## Modelo de Previsão de Sucesso de Startups
---
**Desafio**: Imagine que você foi contratado por uma das maiores aceleradoras do mundo. Sua missão? Desenvolver um modelo preditivo capaz de identificar quais startups apresentam maior probabilidade de se tornarem casos de sucesso no mercado.

**Solução Esperada**: Modelo de machine learning

**Para quem a Solução se Destina**: Investidores e aceleradores na tomada de decisões mais estratégicas

**Dados disponibilizados**: 
- train.csv → conjunto de treinamento com startups + variável de destino (rótulos)
- test.csv → conjunto de teste (sem a coluna de destino)
- sample_submission.csv → formato de submissão esperado

**Metodologia**:

Para desenvolver a solução, a construção de um modelo de previsão de sucesso de startups, é necessário seguir uma sequência de etapas que organize o processo de desenvolvimento. O objetivo é mapear o problema de forma estruturada e compreender qual abordagem de fato poderá solucioná-lo.

Será utilizada a metodologia **CRISP-DM**, adaptada ao desafio. Diferente de um contexto empresarial tradicional, o foco aqui está no entendimento do **problema do desafio**. As etapas são:

1. **Entendimento do contexto** – definição e análise do desafio.  
2. **Exploração dos dados** – levantamento inicial de padrões e inconsistências.  
3. **Preparação dos dados** – limpeza, transformação e formatação adequada.  
4. **Modelagem** – aplicação de algoritmos para previsão de sucesso.  
5. **Avaliação** – análise de métricas e resultados obtidos.  

Ao final, em vez de uma implantação no ambiente de negócio, será realizada a **submissão da solução ao desafio**.


### 1. Entendimento do contexto 

Nesta atividade, cada participante terá acesso ao mesmo conjunto de dados estruturados, representando um problema de previsão real.
O desafio é desenvolver modelos capazes de gerar previsões precisas, que serão avaliadas de acordo com uma métrica específica definida pelos instrutores .

Avaliação:
A avaliação será realizada automaticamente pela plataforma Kaggle, com base nos padrões de submissão definidos no concurso.

Atenção: 
é essencial que todas as submissões sigam rigorosamente o formato exigido, pois qualquer desvio poderá resultar em erros ou desclassificação .

Em situações específicas (como inconsistências, problemas técnicos ou suspeitas de irregularidades), a avaliação pode ser revisada manualmente pelos instrutores.

### 2. Exploração dos Dados

Este conjunto de dados reúne informações reais sobre startups de diferentes setores, incluindo histórico de financiamento, valores captados, localização e setor.

In [ ]:
# Preparação do ambiente: importação de bibliotecas, databases, etc...
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px

print("Pandas:", pd.__version__)
print("Numpy:", np.__version__)
print("Scikit-learn:", sk.__version__)
print("Seaborn:", sns.__version__)
print("Plotly:", px.__version__)
print("Bibliotecas importadas com sucesso!")

print("---")

# Carregamento dos dados
df_train = pd.read_csv("./database/train.csv")
df_test  = pd.read_csv("./database/test.csv")
df_sample_submission = pd.read_csv("./database/sample_submission.csv")

# ================================
# Exploração inicial dos dados
# ================================

def explorar_dataset(df, nome="Dataset", target=None):
    print(f"\n===== {nome} =====")

    # Estrutura básica
    print("\nEstrutura básica")
    print("Formato (linhas, colunas):", df.shape)
    print("Colunas presentes no dataset", df.columns.tolist())
    print("\nTipos de dados:")
    display(df.dtypes.value_counts())
    print("\nInformações de memória:")
    df.info(memory_usage="deep")

    # Qualidade dos dados
    print("\nQualidade dos dados")
    print("Valores ausentes (%):")
    display((df.isna().mean()*100).round(2).sort_values(ascending=False).head(10))

    dup = df.duplicated().sum()
    print(f"Número de duplicatas: {dup}")

    print("\nCardinalidade (nº de valores únicos por coluna):")
    display(df.nunique().sort_values(ascending=False).head(10))

    #Estatísticas descritivas
    print("\nEstatísticas descritivas")
    display(df.describe(include="all").T)

    # Target
    if target and target in df.columns:
        print(f"\nDistribuição da variável alvo '{target}':")
        display(df[target].value_counts(normalize=True).mul(100).round(2))

    # Primeiras linhas para amostra
    print("\nPrimeiras linhas do dataset:")
    display(df.head())


# Chamando a função para cada base
explorar_dataset(df_train, nome="Base de Treinamento", target="labels")
explorar_dataset(df_test, nome="Base de Teste")
explorar_dataset(df_sample_submission, nome="Base de Submissão")


Pandas: 2.2.3
Numpy: 2.1.3
Scikit-learn: 1.6.1
Seaborn: 0.13.2
Plotly: 5.24.1
Bibliotecas importadas com sucesso!
---

===== Base de Treinamento =====

Estrutura básica
Formato (linhas, colunas): (646, 33)
Colunas presentes no dataset ['id', 'age_first_funding_year', 'age_last_funding_year', 'age_first_milestone_year', 'age_last_milestone_year', 'relationships', 'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY', 'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software', 'is_web', 'is_mobile', 'is_enterprise', 'is_advertising', 'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting', 'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB', 'has_roundC', 'has_roundD', 'avg_participants', 'labels']

Tipos de dados:


int64      27
float64     5
object      1
Name: count, dtype: int64


Informações de memória:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate             646 non-null  

age_first_milestone_year    21.36
age_last_milestone_year     17.18
age_first_funding_year       5.42
age_last_funding_year        1.39
id                           0.00
relationships                0.00
funding_rounds               0.00
funding_total_usd            0.00
milestones                   0.00
is_CA                        0.00
dtype: float64

Número de duplicatas: 0

Cardinalidade (nº de valores únicos por coluna):


...

id        277
labels      1
dtype: int64


Estatísticas descritivas


,count,mean,std,min,25%,50%,75%,max
id,277.0,462.98556,271.068618,4.0,215.0,464.0,692.0,922.0
labels,277.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0



Primeiras linhas do dataset:


,id,labels
0,70,0
1,23,0
2,389,0
3,872,0
4,920,0


**Pontos Relevantes Sobre a Exploração**

Colunas do dataset de treino:
- id → identificador
- age_first_funding_year → ano do primeiro financiamento
- age_last_funding_year → ano do último financiamento
- age_first_milestone_year → ano do primeiro marco (milestone)
- age_last_milestone_year → ano do último marco (milestone)
- relationships → relacionamentos
- funding_rounds → rodadas de financiamento
- funding_total_usd → total financiado em USD
- milestones → marcos
- is_CA → está na Califórnia (CA)
- is_NY → está em Nova York (NY)
- is_MA → está em Massachusetts (MA)
- is_TX → está no Texas (TX)
- is_otherstate → está em outro estado
- category_code → código da categoria
- is_software → é do setor de software
- is_web → é do setor de web
- is_mobile → é do setor de mobile
- is_enterprise → é do setor empresarial (enterprise)
- is_advertising → é do setor de publicidade
- is_gamesvideo → é do setor de games/vídeo
- is_ecommerce → é do setor de e-commerce
- is_biotech → é do setor de biotecnologia
- is_consulting → é do setor de consultoria
- is_othercategory → é de outra categoria
- has_VC → possui venture capital (VC)
- has_angel → possui investimento anjo
- has_roundA → possui rodada A
- has_roundB → possui rodada B
- has_roundC → possui rodada C
- has_roundD → possui rodada D
- avg_participants → média de participantes
- labels → rótulos (variável alvo)

### 3. Preparação dos Dados

**Faltantes ( NaN)** : principalmente em age_*e outliers de funding_total_usd→ tratar no pipeline (ex.: imputação por mediana ou uso de modelos robustos). <br>
**Escalas** : variáveis ​​como funding_total_usd, relationships, funding_roundse avg_participantstêm ordens de grandeza diferentes → recomenda-se normalização/padronização ( StandardScaler) em modelos lineares. <br>
**Vazamento evitado** : colunas como status, closed_at, is_top500, dados cruzados e identificadores foram removidos. <br>
**Balanceamento** : verifique a proporção do labelstreinamento; se necessário, use class_weight, Threshold Tuning ou métricas robustas ( AUC/F1).